In [1]:
from grafos import Accion
from grafos import Estado
from grafos import Nodo
from grafos import Problema

Crear métodos auxiliares

In [2]:
def crear_nodo_raiz(problema):
    estado_raiz = problema.estado_inicial
    acciones_raiz = {}
    if estado_raiz.nombre in problema.acciones.keys():
        acciones_raiz = problema.acciones[estado_raiz.nombre]
    raiz = Nodo(estado_raiz, None, acciones_raiz, None)
    return raiz

def crear_nodo_hijo(problema, padre, accion):
    nuevo_estado = problema.resultado(padre.estado, accion)
    acciones_nuevo = {}
    if nuevo_estado.nombre in problema.acciones.keys():
        acciones_nuevo = problema.acciones[nuevo_estado.nombre]
    hijo = Nodo(nuevo_estado, accion, acciones_nuevo, padre)
    costo = padre.costo
    costo += problema.costo_accion(padre.estado, accion)
    hijo.costo = costo
    padre.hijos.append(hijo)
    return hijo

def muestra_solucion(objetivo = None, problema_resolver = None):
    if not objetivo:
        print('No hay solución')
        return
    nodo = objetivo
    while nodo:
        msg = 'Estado: {0}, Costo Total: {1}'
        estado = nodo.estado.nombre
        costo_total = nodo.costo
        print(msg.format(estado, costo_total))
        if nodo.accion:
            accion = nodo.accion.nombre
            padre = nodo.padre.estado
            costo = problema_resolver.costo_accion(padre, nodo.accion)
            msg = "<----{0} [{1}]---->"
            print(msg.format(accion, costo))
            print('-----------')
        nodo = nodo.padre

Crear la función que ejecuta el problema

In [3]:
def costo_uniforme(problema):
    raiz = crear_nodo_raiz(problema)
    frontera = [raiz,]
    explorados = set()
    while True:
        print('frontera: ', [(nodo.estado.nombre, nodo.costo) for nodo in frontera])
        if not frontera:
            return None
        nodo = frontera.pop(0)
        if problema.es_objetivo(nodo.estado):
            return nodo
        print('escoge: ', nodo.estado.nombre)
        print('------------')
        explorados.add(nodo.estado)
        if not nodo.acciones:
            continue
        for nombre_accion in nodo.acciones.keys():
            accion = Accion(nombre_accion)
            hijo = crear_nodo_hijo(problema, nodo, accion)
            estados_frontera = [nodo.estado for nodo in frontera]
            if (hijo.estado not in explorados and hijo.estado not in estados_frontera):
                frontera.append(hijo)
            else:
                buscar = [nodo for nodo in frontera if nodo.estado == hijo.estado]
                if buscar:
                    if hijo.costo < buscar[0].costo:
                        indice = frontera.index(buscar[0])
                        frontera[indice] = hijo
            frontera.sort(key = lambda nodo: nodo.costo)

# Construir estructura del problema

In [4]:
if __name__ == '__main__':
    # vamos a crear las acciones que puede hacer el agente
    accN = Accion('N')
    accS = Accion('S')
    accE = Accion('E')
    accO = Accion('O')
    accNE = Accion('NE')
    accSE = Accion('SE')
    accSO = Accion('SO')
    accNO = Accion('NO')

    # Vamos a crear los estados
    ATec = Estado('ATec', [accSO, accSE])
    CoTec = Estado('CoTec', [accSO, accSE, accE])
    ArqDat = Estado('ArqDat', [accS])
    MesAyu = Estado('MesAyu', [accSE, accE])
    ArqSol = Estado('ArqSol', [accO, accSE])
    GerTec = Estado('GerTec', [accSO, accSE])
    DirAd = Estado('DirAd', [accSE])
    GerenteGeneral = Estado('GerenteGeneral', [])

    # Acciones que puede hacer
    acciones = {'ATec':{'SO':CoTec, 'SE':ArqDat},
                'CoTec':{'SO':MesAyu, 'SE':ArqSol, 'E':ArqDat},
                'ArqDat':{'S':GerTec},
                'MesAyu':{'SE':DirAd, 'E':GerTec},
                'ArqSol':{'O':MesAyu, 'SE':GerTec},
                'GerTec':{'SO':DirAd, 'SE':GerenteGeneral},
                'DirAd':{'SE':GerenteGeneral}
                }
    
    costos = {'ATec':{'SO':10, 'SE':50},
                'CoTec':{'SO':15, 'SE':30, 'E':20},
                'ArqDat':{'S':100},
                'MesAyu':{'SE':150, 'E':40},
                'ArqSol':{'O':10, 'SE':60},
                'GerTec':{'SO':70, 'SE':80},
                'DirAd':{'SE':40}
                }

In [6]:
objetivo = [ GerenteGeneral]
problema = Problema(ATec, objetivo, acciones, costos)
solucion = costo_uniforme(problema)
muestra_solucion(solucion, problema)

frontera:  [('ATec', 0)]
escoge:  ATec
------------
frontera:  [('CoTec', 10), ('ArqDat', 50)]
escoge:  CoTec
------------
frontera:  [('MesAyu', 25), ('ArqDat', 30), ('ArqSol', 40)]
escoge:  MesAyu
------------
frontera:  [('ArqDat', 30), ('ArqSol', 40), ('GerTec', 65), ('DirAd', 175)]
escoge:  ArqDat
------------
frontera:  [('ArqSol', 40), ('GerTec', 65), ('DirAd', 175)]
escoge:  ArqSol
------------
frontera:  [('GerTec', 65), ('DirAd', 175)]
escoge:  GerTec
------------
frontera:  [('DirAd', 135), ('GerenteGeneral', 145)]
escoge:  DirAd
------------
frontera:  [('GerenteGeneral', 145)]
Estado: GerenteGeneral, Costo Total: 145
<----SE [80]---->
-----------
Estado: GerTec, Costo Total: 65
<----E [40]---->
-----------
Estado: MesAyu, Costo Total: 25
<----SO [15]---->
-----------
Estado: CoTec, Costo Total: 10
<----SO [10]---->
-----------
Estado: ATec, Costo Total: 0
